In [2]:
import json
import pandas as pd
from tqdm import tqdm
import re
import string
import nltk
nltk.download('wordnet')
nltk.download('stopwords')
from nltk.corpus import stopwords
stop = stopwords.words('english')
from tqdm import tqdm
import numpy as np
import scann

[nltk_data] Downloading package wordnet to /home/ubuntu/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /home/ubuntu/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
2024-03-01 13:25:33.220034: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-03-01 13:25:33.268450: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-03-01 13:25:33.269218: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-03-01 13:25:33.956349: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [ ]:
result_df = pd.read_csv('../../examples/relevant_jobs_classification/data/score_result_v1.csv')

In [26]:
import requests

def fetch_jobs(job_id):
    
    '''
    Returns Job Data[Hirst, Source : ElasticSearch] for the last_k days, and with flexible page size (<10,000)
    
    Input : Last-K Days, and Page result size
    
    Output : Pandas Dataframe with jobs from the last k days 
    '''
    
    # columns = ['id', 'title','companyStatus', 'min', 'max', 'premium', 'applyCount', 'locations','tags','brandJobFlag']     #Added column name for companyStatus
    # new_cols = ['jobid', 'title','companyStatus', 'min', 'max', 'premium', 'applyCount', 'locations','tags', 'brandJobFlag']       #Added column name for companyStatus
    
    

    url = "http://internal-java-job-searcher-email-backend-1607736061.ap-south-1.elb.amazonaws.com/v1/user/-777/job/jobcode?query=<JOBID>"
    url = url.replace('<JOBID>',job_id)
    payload={}
    headers = {}
    
    response = requests.request("GET", url, headers=headers, data=payload)
    
    # print(response.text)
    

    # url = url.replace("<DAYS>",str(last_k)).replace("<SIZE>",str(size))

    payload={}
    headers = {}

    response = requests.request("GET", url, headers=headers, data=payload)
    job_data = response.json()
    # print(job_data)
    job_df = pd.DataFrame(job_data)

    return job_df[['id','title']]

In [48]:
def clean_space(text):
    return " ".join(re.split("\s+", text, flags=re.UNICODE))

def remove_extra_spaces(input_string):
    # Use regular expression to remove extra spaces (only spaces)
    cleaned_string = re.sub(r' +', ' ', input_string)
    return cleaned_string
     

def get_user_data_search_embed(user_id_list):
    
    '''
    This function combines all the functions for fetching and processing 
    user data from elastic search
    '''


    sub_json = fetch_data_es(user_id_list)
    raw_data = construct_user_data_search_embed(sub_json)

    res_df = pd.DataFrame.from_dict(raw_data, orient='index').T
    result_df = res_df.astype(str).replace("None", " ")
    
    # result_df['resume'] = result_df['resume'].apply(remove_extra_spaces)

    return result_df

def fetch_data_es(user_id_list):
    
    """
    Input : List of Valid Hirist User ID 
    Output : List of User Data JSON stored in Elastic Search
    """
    
    
    response_json = []
    replacements = {'null':'None', 'false':'False'}

    url = "http://10.208.230.226:8080/v1/recruiter/-123/applicant/search/searchById"

    res = []

    for uid in tqdm(user_id_list):


        payload = {
            'usersToSearchFrom' : [str(uid)]
        }

        headers = {
        'Content-Type': 'application/json'
        }

        try:
            response = requests.request("POST", url, headers=headers, data = str(payload))
            response_json = response.json()
    
            res.append(response_json['docs']) 

        except Exception as e:
            continue
        
    return res



def construct_user_data_search_embed(user_json):
    '''
    This is a function for fetching user ID, Keywords (in the form of tags) and 
    user designation from elastic search. This is specifically made for creating 
    user embeddings for our Recruiter search model/POC
    '''
    
    user_dict = {
        'id' : [],
        'keywords': [],
       'current_designation' : [],
        'user_experience':[],  
    
    }
    
    for s in tqdm(user_json):
    
        try:
            source = s[0]
    
            user_dict['id'].append(source['id'])
    
            skill_list = [str(skill['name']).replace(" ","_") for skill in source['skillSet']]
            user_dict['keywords'].append(" ".join(skill_list))
            
            user_dict['user_experience'].append(source['expYear'])
            
            # user_dict['resume'].append(source['resumeText'])
            user_dict['current_designation'].append(source['currentDesignation'])
    
        except Exception as e:
            continue
    
    return user_dict

In [49]:

def get_user_data_in_batches(uid_list, batch_size=1000):
    user_data = pd.DataFrame()

    for i in range(0, len(uid_list), batch_size):
        batch_uids = uid_list[i:i + batch_size]

        batch_user_data = get_user_data_search_embed(batch_uids)

        user_data = pd.concat([user_data, batch_user_data], ignore_index=True)

    return user_data

In [50]:
# def clean_text(input_text):
    
#     '''Make text lowercase, remove text in square brackets,remove links,
#     remove punctuation and remove words containing numbers.'''


#     text = input_text
        
#     text = str(text).lower()
#     text = re.sub('\[.*?\]',' ', text)
#     text = re.sub('https?://\S+|www\.\S+',' ', text)
#     text = re.sub('<.*?>+',' ', text)
#     text = re.sub('[%s]' % re.escape(string.punctuation),' ', text)
#     text = re.sub('\n',' ', text)
#     text = re.sub('\w*\d\w*',' ', text)
#     text = re.sub('yrs',' ',text)
#     text = re.sub('\r',' ',text)
#     text = ' '.join([word for word in text.split() if word not in (stop)])
#     text = re.sub(' +', ' ', text)
    
#     return text

In [51]:
# li = []
# for d in tqdm(main_df['introText']):
#     des = clean_text(d)
#     li.append(des)

In [52]:
# main_df['description']=li

In [53]:
# 

In [54]:
def do_something():
    jid = result_df.job_id.sample().unique()[0]
    r = result_df.loc[result_df['job_id']==jid]
    job_df = fetch_jobs(str(jid))
    uid_list = list(result_df.loc[result_df['job_id']==jid].user_id)
    user_df = get_user_data_in_batches(uid_list)
    return job_df,user_df,r

In [55]:
a,b,c = do_something()
display(a,b,c)

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 37/37 [00:00<00:00, 102232.71it/s]


,id,title
0,1373732,Renaissance Global - Equity Research Analyst -...


,id,keywords,current_designation,user_experience
0,209995,Analyst Financial Investment Equity Stock FMCG...,Research Analyst,8
1,882096,Investment RISK Finance Research CFA Equity Ba...,Associate Vice President,12
2,761462,Investment Financial Research Equity Corporate...,Investment Associate,7
3,2749339,Stock Stata Finance Investments Stock_Market R...,investments and investor relations,0
4,1227,Investment Equity Financial Research Analysis ...,Head of Family Office / Chief Investment Officer,12
5,1596003,Research Equity Financial Equity_Research Anal...,AVP Research,5
6,1017609,Investment Research Analyst CFA Financial Equi...,Equity Research Analyst,6
7,412337,Analyst Financial Research Analytics Valuation...,Assistant Manager (Investment Research),8
8,821531,Investment Research Financial Initiation Equit...,"Manager - CEEMEA Oil & Gas, Middle east utilities",7
9,176058,Financial RISK Research Analysis Trade Investm...,Senior Associate,6


,job_id,description,user_id,resume_text,scores
0,1373732,mumbai based family office looking hire equity...,209995,akshay pradhan address neelkanth sagar mahatma...,0.645710
1,1373732,mumbai based family office looking hire equity...,882096,name mayank kumar gulgulia tel summary cfa usa...,0.690789
2,1373732,mumbai based family office looking hire equity...,761462,sahil nandkumar cfa date birth address flat se...,0.673624
3,1373732,mumbai based family office looking hire equity...,2749339,kushboo jaichand luniya luniya kushboo gmail c...,0.618894
4,1373732,mumbai based family office looking hire equity...,1227,sachin jain mba finance city mumbai contact gm...,0.688748
5,1373732,mumbai based family office looking hire equity...,1596003,sagar shah contact email id gmail com address ...,0.674983
6,1373732,mumbai based family office looking hire equity...,1017609,rahul chandarana c pranay vidya kandivali west...,0.716381
7,1373732,mumbai based family office looking hire equity...,412337,chaitanya vilas bhokare experience management ...,0.508819
8,1373732,mumbai based family office looking hire equity...,821531,dhaval jayant popat email gmail com phone hsbc...,0.447404
9,1373732,mumbai based family office looking hire equity...,176058,vishakha maliwal degree institute location cgp...,0.717085


In [56]:
# d

In [34]:
# list(result_df.loc[result_df['job_id']==jid].user_id)